In [3]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

# Headers for website requests
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:120.0) Gecko/20100101 Firefox/120.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.google.com/",  # Appears as if you're coming from Google
    "Connection": "keep-alive",
    "DNT": "1",  # Do Not Track request
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
}

class Website:

    def __init__(self, url):
        """
        Initialize the website object and fetch content.
        """
        self.url = url
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()  # Raise an error for bad responses
        except requests.RequestException as e:
            print(f'Failed to fetch website: {e}')
            self.title = 'Error fetching website'
            self.text = ''
            return
        
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = 'No readable content found'

url = input('Write down your website URL here: \n')
website = Website(url)

# Model parameters
MODEL = "llama3.2"

# Step 1: Create prompts

system_prompt = (
    "You are an senior research scientist that analyzes the contents of a website \
    and provides a short summary, ignoring text that might be navigation related. \
    Respond in markdown."
)
user_prompt = (
    f"You are looking at a website titled {website.title}. \n\
    The contents of this website is as follows; \
    please provide a short summary of this website in markdown. \
    If it includes news or announcements, then summarize these too.\n\n\
    {website.text}"
)

# Step 2: Construct messages list
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

# Step 3: Call Llama 3.2 and Handle Response
try:
    response = ollama.chat(model=MODEL, messages=messages)
    display(Markdown(response['message']['content']))
except Exception as e:
    print(f'Llama API call failed: {e}')

Write down your website URL here: 
https://www.nlab-utsw.org/


# Website Summary

## Overview
The website appears to be dedicated to the N-LAB, a research organization focused on developing novel neuroimaging and neuroengineering methods. The lab aims to integrate molecular and system neuroscience to solve brain science problems.

## Mission Statement
The mission statement is missing from this snippet, but it mentions understanding neural mechanisms of reward, learning, and memory in rodents' brains.

## Laboratory Details
* Located at the Advanced Imaging Research Center (AIRC), Department of Neuroscience, and Peter O'Donnel Brain Institute at UT Southwestern Medical Center.
* Strong support from AIRC provides access to state-of-the-art resources for imaging modalities.
* Supported by the Endowed Scholar Program, with access to 18 core facilities on campus.

## News/Anotations
No recent news or announcements appear in this snippet. However, there is a "Learn more" link that suggests additional information about the lab, its team, and research activities.

## Navigation Links
Navigation links:
* Research
* Publications
* People
* Gallery
* Bio-ART
* Contact

Note: The "HOME", "RESEARCH", "PUBLICATIONS", etc. links appear to be redundant and do not provide meaningful content.